# S2orc (exploration, clustering & visualization)
---
---
For presenting some results we need to analyze (and rapidly compare) some of the methods we used untill now in order to discriminates between paper's `field_of_study` based on their `title` and `abstract`.
This notebook is an extention of some previous work done by Master's students from University of Florence (cite here).

## Dataset

From each scientific paper we took the `title` and the `abstract`, as well as a property identifying the field in witch the article pertrains.
The dataset (only 1000 elements) has been selected randomly from a full-version of 80M papers from different fields.
The field of studies (that are called in the dataset `mag_field_of_study`) are the following:

| Field of study | All papers | Full text |
|----------------|------------|-----------|
| Medicine       | 12.8M      | 1.8M      |
| Biology        | 9.6M       | 1.6M      |
| Chemistry      | 8.7M       | 484k      |
| n/a            | 7.7M       | 583k      |
| Engineering    | 6.3M       | 228k      |
| Comp Sci       | 6.0M       | 580k      |
| Physics        | 4.9M       | 838k      |
| Mat Sci        | 4.6M       | 213k      |
| Math           | 3.9M       | 669k      |
| Psychology     | 3.4M       | 316k      |
| Economics      | 2.3M       | 198k      |
| Poli Sci       | 1.8M       | 69k       |
| Business       | 1.8M       | 94k       |
| Geology        | 1.8M       | 115k      |
| Sociology      | 1.6M       | 93k       |
| Geography      | 1.4M       | 58k       |
| Env Sci        | 766k       | 52k       |
| Art            | 700k       | 16k       |
| History        | 690k       | 22k       |
| Philosophy     | 384k       | 15k       |


Note for reproducibility: `data` is a `DatasetDict` object composed by `Dataset` object for every key (in `train`, `test`, `valid`):

```python
{ 
    "train": Dataset,
    "test" : Dataset,
    "valid": Dataset
}
```

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
MAIN_PATH = '/home/vivoli/Thesis'
DATA_PATH = '/home/vivoli/Thesis/data'
OUT_PATH  = '/home/vivoli/Thesis/outputs/'
ARGS_PATH = '/home/vivoli/Thesis/'

In [3]:
# Imports

from thesis.utils.general import load_dataset_wrapper
from thesis.utils.parsers.args_parser import parse_args

DICTIONARY_FIELD_NAMES = dict(
    train         = ['train'],
    test          = ['test', 'debug', 'dev'],
    validation    = ['validation', 'valid']
)

## Getting the dataset
---
In order to get the dataset we need to create a dictionary with the DatasetArguments (params) and use our "library" called `thesis`.

In [14]:
# ------------------
# Creating Arguments
# ------------------

# create arguments dictionary
args = dict(
    
    # DatasetArguments
    model_name_or_path           = "allenai/scibert_scivocab_uncased",
    dataset_name                 = "s2orc", # "keyphrase",
    dataset_config_name          = "full",  # "inspec",

    # TrainingArguments        
    seed                         = '1234', 
    output_dir                   = "/home/vivoli/Thesis/output",
    
    num_train_epochs             = '1',
    per_device_train_batch_size  = "8", # 16 and 32 end with "RuntimeError: CUDA out of memory."
    per_device_eval_batch_size   = "8", # 16 and 32 end with "RuntimeError: CUDA out of memory."
    max_seq_length               = '512',

    # S2orcArguments & KeyPhArguments
    dataset_path                 = "/home/vivoli/Thesis/data",
    data                         = "abstract",
    target                       = "title",             
    classes                      = "mag_field_of_study", # "keywords",

    # S2orcArguments
    idxs                         = '0',
    zipped                       = 'True',
    mag_field_of_study           = "Computer Science",    
    keep_none_papers             = 'False',
    keep_unused_columns          = 'False',

    # RunArguments
    run_name                     = "scibert-s2orc",
    run_number                   = '0',
    run_iteration                = '0',

    # LoggingArguments
    verbose                      = 'True',
    debug_log                    = 'True',
    time                         = 'False',
    callbacks                    = "WandbCallback,CometCallback,TensorBoardCallback",
)

# save dictionary to file
import json
import os

ARGS_FILE = 'arguments.json'

with open(os.path.join(ARGS_PATH, ARGS_FILE), 'w') as fp:
    json.dump(args, fp)
    
print(args)

{'model_name_or_path': 'allenai/scibert_scivocab_uncased', 'dataset_name': 's2orc', 'dataset_config_name': 'full', 'seed': '1234', 'output_dir': '/home/vivoli/Thesis/output', 'num_train_epochs': '1', 'per_device_train_batch_size': '8', 'per_device_eval_batch_size': '8', 'max_seq_length': '512', 'dataset_path': '/home/vivoli/Thesis/data', 'data': 'abstract', 'target': 'title', 'classes': 'mag_field_of_study', 'idxs': '0', 'zipped': 'True', 'mag_field_of_study': 'Computer Science', 'keep_none_papers': 'False', 'keep_unused_columns': 'False', 'run_name': 'scibert-s2orc', 'run_number': '0', 'run_iteration': '0', 'verbose': 'True', 'debug_log': 'True', 'time': 'False', 'callbacks': 'WandbCallback,CometCallback,TensorBoardCallback'}


In [17]:
# ------------------
# Parsing the Arguments
# ------------------

dataset_args, training_args, model_args, run_args, log_args, embedding_args = parse_args(['params_path', os.path.join(ARGS_PATH, ARGS_FILE)])

/home/vivoli/miniconda3/envs/thesis/lib/python3.6/site-packages/ipykernel_launcher.py
from notebook: True
argv: ['params_path', '/home/vivoli/Thesis/arguments.json']
s2orc


In [ ]:
# ------------------
# Getting the datasets
# ------------------

# Getting the load_dataset wrapper that manages huggingface dataset and the custom ones
custom_load_dataset = load_dataset_wrapper()
# Loading the raw data based on input (and default) values of arguments
raw_datasets = custom_load_dataset(dataset_args, training_args, model_args, run_args, log_args, embedding_args)


# The Datasets in the raw form can have different form of key names (depending on the configuration).
# We need all datasets to contain 'train', 'test', 'validation' keys, if not we change the dictionary keys' name
# based on the `names_tuple` and conseguently on `names_map`.
def format_key_names(raw_datasets):
    # The creation of `names_map` happens to be here
    # For every element in the values lists, one dictionary entry is added 
    # with (k,v): k=Value of the list, v=Key such as 'train', etc.
    def names_dict_generator(names_tuple: dict):
        names_map = dict()
        for key, values in names_tuple.items():
            for value in values:
                names_map[value] = key
        return names_map
    names_map = names_dict_generator(DICTIONARY_FIELD_NAMES)
    split_names = raw_datasets.keys()
    for split_name in split_names:
        new_split_name = names_map.get(split_name)
        if split_name != new_split_name:
            raw_datasets[new_split_name] = raw_datasets.pop(split_name)  
    return raw_datasets

logger.info(f"Formatting DatasetDict keys")
datasets = format_key_names(raw_datasets)

In [ ]:
keywords = []
keywords_info = {}
for item in data: 
  temp = item['keywords']
  for keyword in temp:
    keyword = keyword.replace("-", "").replace(",","").replace("/", "") #mi ero scordato di togliere il trattino nel preprocessing delle keywords. Per la virgola, non è un separatore, ma è una keyword che ha la virgola, e.g. "segmentation, features and descriptions"
    if keyword not in keywords: 
      keywords.append(keyword)
      keywords_info[keyword] = {'count': 0, 'appears_in': []}
    keywords_info[keyword]['count'] += 1
    keywords_info[keyword]['appears_in'].append(item['filename'])

print(keywords_info)

In [ ]:
#plot distribution
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.pyplot as plt; plt.rcdefaults()


pos = np.arange(len(keywords))
counts = []
for kw in keywords:
  counts.append(keywords_info[kw]['count'])

plt.figure(figsize=(10,25))
y_pos = np.arange(len(keywords))
plt.barh(y_pos, counts, alpha=0.5)
plt.yticks(y_pos, keywords)
plt.xlabel('Count')
plt.title('Count distribution for each keyword')
plt.grid()

plt.show()


In [ ]:
#order by count
ordered_kws = [x for _,x in sorted(zip(counts,keywords))]
ordered_cts = sorted(counts)


plt.figure(figsize=(5,22))
y_pos = np.arange(len(keywords))
plt.barh(y_pos, ordered_cts, alpha=0.5)
plt.yticks(y_pos, ordered_kws)
plt.xlabel('Count')
plt.title('Count distribution for each keyword')
#plt.grid()

plt.show()

Given the chart of frequency of the keywords (NOT normalized), a threshold can be set to only consider the most relevant keywords

# Definition of groups for the "ground truth" (?) "baseline" (?)


Note: the following is arguable; in fact, the most frequent keywords are also the blandest and, maybe, less significant for a categorization. 

In [ ]:
len(keywords)

In [ ]:
len(keywords)*.2 #the first 29 words make up to 20% of all the keywords

In [ ]:
sum(ordered_cts[len(ordered_cts)-43:len(ordered_cts)])/sum(ordered_cts)

In [ ]:
sum(ordered_cts[len(ordered_cts)-29:len(ordered_cts)])/sum(ordered_cts)

43 keywords make up roughly 80% of the total keywords count. However, for practical reasons, 29 keywords (20%) will be used, also considering how many documents have multiple keywords associated to them.
This will have consequences in the choice of the number of clusters i.e. if HDBSCAN doesn't need nor wants the number of clusters to be specified, when using k-means it is mandatory for the nature of the algorithm. 

In [ ]:
MOST_IMPORTANT_KW_THRESHOLD = 29

In [ ]:
mi_keywords = ordered_kws[len(ordered_kws)-MOST_IMPORTANT_KW_THRESHOLD:len(ordered_kws)] #most important keywords
mi_keywords_counts = ordered_cts[len(ordered_kws)-MOST_IMPORTANT_KW_THRESHOLD:len(ordered_kws)]
plt.rc('font', size=8)
plt.figure(figsize=(10,3))
y_pos = np.arange(len(mi_keywords))
plt.barh(y_pos, mi_keywords_counts, alpha=0.5)
plt.yticks(y_pos, mi_keywords, )
plt.xlabel('Count')
plt.title('Count distribution for each of the most important keywords')

plt.grid()

plt.show()

In [ ]:
mi_associations = {}
#direi molto poco elegante ma okay
for keyword in mi_keywords: 
  mi_associations[keyword] = keywords_info[keyword]

print(mi_associations['deeplearning'])

# Clustering
From here on clustering will be considered using SBERT embeddings. 
The variable called 'clustering_on' is used to discriminate and chose weather the embeddings are made on the abstracts or on the titles. In both cases, clustering is made through HDBSCAN and k-means (see the two subsections).
WARNING: do not execute cells in random order. Some variables names are used both for the HDBSCAN clustering and for k-means; the suggestion is to execute hdbscan first, and k-means second. Otherwise, it is important to execute the definition of the functions used for both cases. 

## Clustering with HDBSCAN

In [ ]:
!pip install torch==1.7.1+cu101 torchvision==0.8.2+cu101 torchaudio==0.7.2 -f https://download.pytorch.org/whl/torch_stable.html
!pip install hdbscan
!pip install sentence-transformers
!pip install umap-learn

In [ ]:
from sentence_transformers import SentenceTransformer
#model_in = SentenceTransformer('distilbert-base-nli-mean-tokens') #this model is kinda bad
model_in = SentenceTransformer('stsb-roberta-large')
#model = SentenceTransformer('paraphrase-distilroberta-base-v1')

In [ ]:

#since the same exact thing has to be done for both abstracts and titles, i define a function
def elaborate(subject = None, model = None): 
  '''
  :param str subject: can be either 'abstract' or 'title', specifies what the clustering has to be made on 
  :param SentenceTransformer model: instanciated model of SentenceTransformer (SBERT)
  '''
  textual_data = []

  for item in data:
    # the condition is rather importante. In the case of abstracts, it kept clustering 
    # (with HDBSCAN) all the error in the same cluster (which makes sense), remortion is thereby necessary
    if not (item[subject] == 'UNABLE TO DECODE ABSTRACT' or item[subject] == 'Unable to open the pdf' or item[subject] == ""): 
      textual_data.append(item[subject])
  print(textual_data)
  return textual_data, model.encode(textual_data, show_progress_bar = True)




In [ ]:
#funzione, dati dati originali e label li stampa assieme

def constructDictionaryOfClusters(labels, original_data):
  print(labels)
  associative = []
  #prepare dictionary
  
  for i in range(labels.max()+1): 
    associative.append([]) 
  #print(f"associative: {associative}")

  for i in range(len(original_data)):
    if labels[i] != -1: #in the case of HDBSCAN the labels -1 are the "outsiders"
      associative[labels[i]].append(original_data[i])
    #print(i)

  for item in associative:
    print(len(item))
    print(item)
  print(len(associative))
  return associative
  #nota: l'ultimo elemento della lista è composto dagli outsiders

In [ ]:
clustering_on = "abstract"

In [ ]:
textual_data, embeddings = elaborate(clustering_on, model_in)

#########################
#instanciate model
embeddings = model.encode(textual_data, show_progress_bar = True)


In [ ]:
print(embeddings.shape)

In [ ]:
#here after, once calculated the embeddings (either for the abstracts or the title) the clustering is considered
#first with HDBSCAN, then k-means
import umap
import hdbscan
# first it's *better* to do dimensionality reduction, sbert returns embeddings of dimension 700+ or even 1000+ (depending on the chosen model)
# clustering algorithms seem not to perform well for for big dimensions use-cases
# UMAP is a popular algorithm for dimensionality reduction

umap_embeddings = umap.UMAP(n_neighbors=15, 
                            n_components=5, 
                            metric='cosine').fit_transform(embeddings)

cluster = hdbscan.HDBSCAN(min_cluster_size=5,
                          metric='euclidean',                      
                          cluster_selection_method='eom').fit(umap_embeddings)

In [ ]:
import pandas as pd
umap_data = umap.UMAP(n_neighbors=15, n_components=2, min_dist=0.0, metric='cosine').fit_transform(umap_embeddings)
result = pd.DataFrame(umap_data, columns=['x', 'y'])
result['labels'] = cluster.labels_

# Visualize clusters
fig, ax = plt.subplots(figsize=(10, 7))
outliers = result.loc[result.labels == -1, :]
clustered = result.loc[result.labels != -1, :]
plt.scatter(outliers.x, outliers.y, color='#BDBDBD', s=0.05)
plt.scatter(clustered.x, clustered.y, c=clustered.labels, s=0.05, cmap='hsv_r')
plt.colorbar()
plt.title("Visualization of one instance of clustering through HDBSCAN")

In [ ]:
'''cluster = hdbscan.HDBSCAN(min_cluster_size=3,
                          metric='manhattan',                      
                          cluster_selection_method='eom').fit(embeddings)
'''

In [ ]:
#number of clusters
print(cluster.labels_.max()) #nota: è il label più alto => #numero clusters = max()+1

In [ ]:
clusters = constructDictionaryOfClusters(cluster.labels_, textual_data)

In [ ]:
print(len(clusters))

In [ ]:
#really not necessary to be honest, 
clusters_desc = clusters.copy()[:-1]
clusters_desc.sort(key = len, reverse = True)

In [ ]:
print(len(clusters))
print(len(clusters_desc))

Now I do sort of a matching matrix. Maybe it would be appropriate to do intersection over union for each element of the matrix?

In [ ]:
#transform clusters of titles/abstracts into clusters of ids
'''clusters_of_ids = []

for item in clusters_desc:
  temp = []
  for text in item: 
    tmp_ids = []
    for dt in data: 
      #print(dt)
      tp = None
      if dt[clustering_on] == text: 
        tmp_ids.append(dt['filename'])
        tp = dt['filename']
        #print(tp)
        break
    temp.append(tp)
         
  clusters_of_ids.append(temp)
#print(clusters_of_ids)
'''

In [ ]:
def transform_clusters_into_id_clusters(temp_clust_desc):
  id_clusters = []

  for item in clusters_desc:
    temp = []
    for text in item: 
      tmp_ids = []
      for dt in data: 
        #print(dt)
        tp = None
        if dt[clustering_on] == text: 
          tmp_ids.append(dt['filename'])
          tp = dt['filename']
          #print(tp)
          break
      temp.append(tp)
          
    id_clusters.append(temp)
  return id_clusters

In [ ]:
clusters_of_ids = transform_clusters_into_id_clusters(clusters_desc)

In [ ]:
print(len(clusters_of_ids[0]))
print(len(clusters_of_ids[-1]))

In [ ]:
#should have stored this way since the beginning, useless transformation..
new_data = {}
for item in data: 
  new_data[item['filename']] = {'title': item['title'], 'abstract': item['abstract']}

In [ ]:
#new_data

In [ ]:
mi_keywords_desc = mi_keywords.copy()
mi_keywords_desc.reverse()
matching_matrix = []

#for each cluster
for c_item in clusters_of_ids: 
  #for each keyword of the previously defined keywords
  row = []
  for kwd in mi_keywords_desc:
    # keyword k_item appears in doc1, doc2, ...
    # c_item is the first set, the second set should be k_item['appears_in']
    appears_in = mi_associations[kwd]['appears_in']
    #the following can be replaced with whatever metric
    union = len(set(c_item).union(set(appears_in)))
    intersection = len(set(c_item).intersection(set(appears_in)))
    row.append(intersection/union)
  matching_matrix.append(row)

import numpy as np
np_matching_matrix = np.array(matching_matrix)
#print(np_matching_matrix)

In [ ]:
mi_keywords_desc

In [ ]:
import seaborn as sn
#TODO add the labels to the chart ?

plt.figure(figsize=(11,8))

sn.heatmap(np_matching_matrix, annot=False, xticklabels=mi_keywords_desc)

## Clustering with K-means

The reason for doing two types of clustering is that, while HDBSCAN leaves out the outsideres, k-means forces each element into a cluster. 

In [ ]:
from sklearn.cluster import KMeans
clustering_model = KMeans(n_clusters=29)
clustering_model.fit(umap_embeddings)
cluster_assignment = clustering_model.labels_
# number of clusters
print(clustering_model.labels_.max())

clusters = constructDictionaryOfClusters(clustering_model.labels_, textual_data)


In [ ]:
import pandas as pd
#umap_data = umap.UMAP(n_neighbors=15, n_components=2, min_dist=0.0, metric='cosine').fit_transform(umap_embeddings)
result = pd.DataFrame(umap_data, columns=['x', 'y'])
result['labels'] = clustering_model.labels_

# Visualize clusters
fig, ax = plt.subplots(figsize=(10, 7))
outliers = result.loc[result.labels == -1, :]
clustered = result.loc[result.labels != -1, :]
plt.scatter(outliers.x, outliers.y, color='#BDBDBD', s=0.05)
plt.scatter(clustered.x, clustered.y, c=clustered.labels, s=0.05, cmap='hsv_r')
plt.colorbar()
plt.title("Visualization of one instance of clustering through k-means")

In [ ]:
#really not necessary to be honest, 
clusters_desc = clusters.copy()[:-1]
clusters_desc.sort(key = len, reverse = True)

NOTE: from this point on, it's the same code as with HDBSCAN

In [ ]:
clusters_of_ids = transform_clusters_into_id_clusters(clusters_desc)
print(len(clusters_of_ids[0]))
print(len(clusters_of_ids[-1]))

In [ ]:
#should have stored this way since the beginning, useless transformation..
new_data = {}
for item in data: 
  new_data[item['filename']] = {'title': item['title'], 'abstract': item['abstract']}

In [ ]:
mi_keywords_desc = mi_keywords.copy()
mi_keywords_desc.reverse()
matching_matrix = []

#for each cluster
for c_item in clusters_of_ids: 
  #for each keyword of the previously defined keywords
  row = []
  for kwd in mi_keywords_desc:
    # keyword k_item appears in doc1, doc2, ...
    # c_item is the first set, the second set should be k_item['appears_in']
    appears_in = mi_associations[kwd]['appears_in']
    #the following can be replaced with whatever metric
    union = len(set(c_item).union(set(appears_in)))
    intersection = len(set(c_item).intersection(set(appears_in)))
    row.append(intersection/union)
  matching_matrix.append(row)

import numpy as np
np_matching_matrix = np.array(matching_matrix)
#print(np_matching_matrix)

In [ ]:
import seaborn as sn
#TODO add the labels to the chart ?

plt.figure(figsize=(11,8))
sn.heatmap(np_matching_matrix, annot=False, xticklabels=mi_keywords_desc)

# A further step: automatic keyword assignment
Given the clusters, it's possibile tu use c-
TF-IDF to infer the topic, this *could* allow for automatic labeling of a set of documents

In [ ]:
#smarter way of doing things..
import pandas as pd
#for each cluster, create pandas dataframe 
docs_df = pd.DataFrame(textual_data, columns=["Doc"])
docs_df['Topic'] = cluster.labels_
docs_df['Doc_ID'] = range(len(docs_df))
docs_per_topic = docs_df.groupby(['Topic'], as_index = False).agg({'Doc': ' '.join})

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
#note: c-tf-idf is simply tf-idf but the measurements are made on one entire cluster
def c_tf_idf(documents, m, ngram_range=(1, 1)):
    count = CountVectorizer(ngram_range=ngram_range, stop_words="english").fit(documents)
    t = count.transform(documents).toarray()
    w = t.sum(axis=1)
    tf = np.divide(t.T, w)
    sum_t = t.sum(axis=0)
    idf = np.log(np.divide(m, sum_t)).reshape(-1, 1)
    tf_idf = np.multiply(tf, idf)

    return tf_idf, count
  

In [ ]:
def extract_top_n_words_per_topic(tf_idf, count, docs_per_topic, n=20):
    words = count.get_feature_names()
    labels = list(docs_per_topic.Topic)
    tf_idf_transposed = tf_idf.T
    indices = tf_idf_transposed.argsort()[:, -n:]
    top_n_words = {label: [(words[j], tf_idf_transposed[i][j]) for j in indices[i]][::-1] for i, label in enumerate(labels)}
    return top_n_words

def extract_topic_sizes(df):
    topic_sizes = (df.groupby(['Topic'])
                     .Doc
                     .count()
                     .reset_index()
                     .rename({"Topic": "Topic", "Doc": "Size"}, axis='columns')
                     .sort_values("Size", ascending=False))
    return topic_sizes



In [ ]:
tf_idf, count = c_tf_idf(docs_per_topic.Doc.values, m=len(data))
top_n_words = extract_top_n_words_per_topic(tf_idf, count, docs_per_topic, n=20)
topic_sizes = extract_topic_sizes(docs_df) 
topic_sizes.head(10) #nota, i topic con '-1' sono quelli che hdbscan non ha clusterizzato

In [ ]:
top_n_words[0][:100]